# デンドログラム
デンドログラム作成には次を使用<br>
https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html<br>
https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html

- ここで用いたデータセットは，総務省統計局https://www.e-stat.go.jp/ において，“地域別１世帯当たり１か月間の収入と支出 ”　“総世帯”を検索ー＞「平成26年全国消費実態調査 / 全国 / 家計収支に関する結果 / 総世帯」の[EXCEL] をクリックするとファイル"ra01301.xls"（2023年4月現在のファイル名）がダウンロードされる。このファイルはフォルダ内に置き，読者に提供
- 手作業で加工し，「食品小分類」と都道府県だけを抽出し，これをスクリプト中のCSVファイル名で提供
- ただし，食品小分類の酒類のデータは，元データの大分類をそのまま流用

#### 分析方針は次の文献を参照：
[1] 第 2 講 - 総務省統計局　https://www.stat.go.jp/teacher/dl/pdf/c3learn/materials/third/dai2.pdf

#### Warning回避
次のスクリプトは，下記のWariningを回避するためのものです。ただし，このWariningは無視しても良いので，次のスクリプトの実行は必須ではありません。

- UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
- Ref.:https://stackoverflow.com/questions/69596239/how-to-avoid-memory-leak-when-dealing-with-kmeans-for-example-in-this-code-i-am

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = '1'

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from scipy.cluster import hierarchy
from sklearn.cluster import KMeans

from sklearn.decomposition import PCA

import seaborn as sns
import japanize_matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [ ]:
df = pd.read_csv('dataFoodExpenditure.csv', comment='#')
df.head()

In [ ]:
print(df.columns) # ヘッダーの表示
print(df.index) #インデックスの表示

In [ ]:
pref_labels = list(df['都道府県']) # 都道府県名のリスト，prefecture list, dendrogramへの入力用。
len(pref_labels)

In [ ]:
df2 = df.drop('都道府県', axis='columns')
df2.head()

In [ ]:
fig = plt.subplots(figsize=(14,4))
Z = hierarchy.linkage(df2, method='ward')
dn = hierarchy.dendrogram(Z, labels=pref_labels, leaf_rotation=90., leaf_font_size=14)
#plt.savefig('fig_ClusterAggr_FoodExpend.pdf', bbox_inches='tight')
plt.show()

### 主成分分析
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html<br>
https://ja.wikipedia.org/wiki/主成分分析<br>
https://www.nttcoms.com/service/research/dataanalysis/principal-component-analysis/

In [ ]:
pca = PCA()
pca.fit(df2)
feature = pca.transform(df2)
#feature.shape

In [ ]:
fig = plt.subplots(figsize=(10,8))
for (x, y, label) in zip(feature[:,0], feature[:,1], pref_labels):
    plt.scatter(x, y, alpha=0.8, c="w", edgecolors='k')
#    plt.annotate(label, xy=(x,y))
    plt.text(x,y, label, fontsize=14)

plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")

### kMeans
デンドログラムを見て，クラスタ数を決めた

In [ ]:
kmeans = KMeans(n_clusters=7, n_init='auto')
y_pred = kmeans.fit_predict(df2)

Choosing Colormaps in Matplotlib https://matplotlib.org/stable/tutorials/colors/colormaps.html

In [ ]:
fig  = plt.subplots(figsize=(12,6))
plt.scatter(feature[:,0], feature[:,1], c=y_pred, s=80, cmap=cm.Accent, edgecolors='k', )
plt.colorbar()

for i, label in enumerate(pref_labels):
    plt.text(feature[i,0], feature[i,1], label, fontsize=12)
    

plt.xlabel('第一主成分', fontsize=14)
plt.ylabel('第二主成分', fontsize=14)
plt.grid()
#plt.savefig('fig_PCA_kMeans_FoodExpend.pdf', bbox_inches='tight')
plt.show()

上記の考察は，参考文献[1]のp.14を参照されたい。

#### 食品小分類間の相関行列のヒートマップ表示
念のため，相関行列を見て，上記のクラスタリング結果の考察に役立ちそうなものが有るかを見る。あまり，有力な手掛かりは見当たらない。

In [ ]:
corr_df = df2.corr(method='pearson')
#corr_df
fig = plt.subplots(figsize=(14,10))
sns.heatmap(corr_df, annot=True)
plt.show()